# Download responses and analyze them with DuckDB

In [1]:
%pip install --upgrade pip duckdb-engine faker sqlalchemy jupysql --quiet
%pip install ../.. --quiet  # or pip install citric

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path

from faker import Faker

import citric

## Create citric client

In [3]:
# Use your own server's parameters here
client = citric.RPC(
    "http://localhost:8001/index.php/admin/remotecontrol",
    "iamadmin",
    "secret",
)

## Load survey file

In [4]:
with Path("../../examples/survey.lss").open("rb") as f:
    survey_id = client.import_survey(f)

## Generate fake response data

In [5]:
fake = Faker()

fake_responses = []

for _ in range(100):
    start_date = fake.date_time_this_month()
    submit_date = fake.date_time_between_dates(start_date)

    response = {
        "G01Q01": fake.text(max_nb_chars=100),
        "G01Q02": fake.random_int(1, 5),
        "token": fake.sha1()[:5],
        "ipaddr": fake.ipv4(),
        "startdate": start_date.isoformat(),
        "submitdate": submit_date.isoformat(),
    }

    fake_responses.append(response)

len(fake_responses)

100

## Activate survey and load fake responses

In [6]:
client.activate_survey(survey_id)
client.activate_tokens(survey_id)

result = client.add_responses(survey_id, fake_responses)
len(result)

100

## Export responses to a CSV file

In [7]:
with Path("responses.csv").open("wb") as file:
    file.write(client.export_responses(survey_id, file_format="csv"))

## Import the CSV into DuckDB

In [8]:
%load_ext sql

Found pyproject.toml from '/Users/edgarramirez/Code/edgarrmondragon/citric'

In [9]:
%sql duckdb://

In [10]:
%sql CREATE TABLE responses AS SELECT * FROM 'responses.csv'

Running query in 'duckdb://'

Count


## Query the responses with DuckDB

In [11]:
%sql SELECT * FROM responses LIMIT 5

Running query in 'duckdb://'

id,submitdate,lastpage,startlanguage,seed,token,startdate,datestamp,ipaddr,G01Q01,G01Q02,G02Q03,G02Q03[filecount]
1,2023-08-07 01:01:39,None,en,None,b468e,2023-08-01 21:15:42,2023-08-08 19:56:29,182.63.148.172,Him walk white senior win send contain.,3,None,None
2,2023-08-06 13:53:54,None,en,None,9bc69,2023-08-03 12:59:20,2023-08-08 19:56:29,53.181.76.129,Perform bar evidence hour inside safe.,3,None,None
3,2023-08-06 01:52:39,None,en,None,e1c82,2023-08-03 09:37:28,2023-08-08 19:56:29,4.124.173.51,Win weight remain tough factor thing none. Within boy country develop no sure dream.,4,None,None
4,2023-08-07 00:51:37,None,en,None,f3c0c,2023-08-04 19:39:18,2023-08-08 19:56:29,155.33.200.9,Within far deep. Need however he purpose reach. Why one adult return create such spring he.,2,None,None
5,2023-08-03 05:06:15,None,en,None,93832,2023-08-01 07:00:08,2023-08-08 19:56:29,163.127.182.180,Level Democrat under smile check.,4,None,None


### Show tally for multiple choice question

In [12]:
%sql SELECT G01Q02, count(1) as TOTAL FROM responses GROUP BY G01Q02

Running query in 'duckdb://'

G01Q02,TOTAL
1,20
2,16
3,23
4,24
5,17


### Show which respondents took the longest to submit their responses

In [13]:
%%sql
SELECT token, submitdate - startdate AS duration
FROM responses
ORDER BY 2 DESC LIMIT 10

Running query in 'duckdb://'

token,duration
44288,"5 days, 18:58:30"
70c3e,"5 days, 8:48:22"
69491,"5 days, 8:01:47"
b468e,"5 days, 3:45:57"
fde2a,"4 days, 22:14:05"
42bc4,"4 days, 19:00:15"
9c08f,"4 days, 17:08:00"
bcade,"4 days, 12:04:27"
0bcd7,"4 days, 8:41:31"
993ee,"4 days, 7:21:23"
